In [62]:
## this code adds the OS speeds to the OSM data where possible
## you'll need to have joined the speed data to a copy of OS MasterMap roads

In [1]:
import geopandas as gpd

# get OS roads with speed data
file_path = r'C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\GIS Dataset\newcastle_roads_withavgspeeds.gpkg'
os_roads = gpd.read_file(file_path)

os_roads = os_roads.to_crs('4326')

In [2]:
# get OSM roads
import osmnx as ox

place = "Newcastle Upon Tyne, United Kingdom"
g = ox.graph_from_place(place, network_type='drive')

# Convert graph to GeoDataFrames
nodes_osm, edges_osm = ox.graph_to_gdfs(g)

In [6]:
print(len(edges_osm))
print(len(os_roads))

22941
30216


In [7]:
# Perform spatial join
joined_data = gpd.sjoin_nearest(edges_osm, os_roads, how='left')

# drop duplicates, taking first value
joined_data = joined_data[~joined_data.geometry.duplicated()]

print(len(joined_data))

c:\Users\b8008458\Anaconda3\envs\ox_151\Lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


22941


In [11]:
joined_data

osmid  \
u           v           key                                                      
6565642     13800629    0                                              2953792   
6565645     13800665    0                               [921085898, 194663139]   
6565653     13801221    0                                 [921446883, 8125391]   
6565679     13800639    0    [135903555, 135903561, 38061098, 38061099, 161...   
            447588156   0                      [921061592, 921061591, 2953839]   
...                                                                        ...   
11879532996 11879532998 0                                           1279716767   
11879532998 11879532982 0                                           1279716767   
            250208162   0                                           1279716769   
11879533019 1541333358  0                                            459143891   
            2889011248  0                    [935302208, 738932787, 935302206]   

                             oneway      lanes  ref       highway maxspeed  \
u           v           key                                                  
6565642     13800629    0      True          3   A1         trunk   60 mph   
6565645     13800665    0      True          3   A1         trunk   60 mph   
6565653     13801221    0      True          1   A1    trunk_link   70 mph   
6565679     13800639    0      True          2   A1         trunk   70 mph   
            447588156   0      True  [3, 1, 2]  NaN    trunk_link   70 mph   
...                             ...        ...  ...           ...      ...   
11879532996 11879532998 0      True        NaN  NaN  unclassified      NaN   
11879532998 11879532982 0      True        NaN  NaN  unclassified      NaN   
            250208162   0      True          1  NaN  unclassified      NaN   
11879533019 1541333358  0      True          2  NaN      tertiary   30 mph   
            2889011248  0     False        NaN  NaN      tertiary   30 mph   

                            reversed  length_left  \
u           v           key                         
6565642     13800629    0      False      612.471   
6565645     13800665    0      False      489.465   
6565653     13801221    0      False      633.060   
6565679     13800639    0      False     1073.101   
            447588156   0      False      515.087   
...                              ...          ...   
11879532996 11879532998 0      False        5.937   
11879532998 11879532982 0      False       10.194   
            250208162   0      False       14.815   
11879533019 1541333358  0      False       10.365   
            2889011248  0       True      202.516   

                                                                      geometry  \
u           v           key                                                      
6565642     13800629    0    LINESTRING (-1.68446 54.99226, -1.68660 54.990...   
6565645     13800665    0    LINESTRING (-1.67287 55.00194, -1.67570 54.999...   
6565653     13801221    0    LINESTRING (-1.69809 54.97782, -1.69796 54.977...   
6565679     13800639    0    LINESTRING (-1.62902 55.03969, -1.62883 55.039...   
            447588156   0    LINESTRING (-1.62902 55.03969, -1.62832 55.038...   
...                                                                        ...   
11879532996 11879532998 0    LINESTRING (-1.66569 55.01286, -1.66570 55.012...   
11879532998 11879532982 0    LINESTRING (-1.66574 55.01281, -1.66578 55.012...   
            250208162   0    LINESTRING (-1.66574 55.01281, -1.66578 55.012...   
11879533019 1541333358  0    LINESTRING (-1.66668 55.01330, -1.66670 55.01320)   
            2889011248  0    LINESTRING (-1.66668 55.01330, -1.66661 55.013...   

                            bridge  ... merged_data_SS_10-14A  \
u           v           key         ...                         
6565642     13800629    0      NaN  ...                  56.5   
6565645     13800665    0      NaN  ...

In [16]:
for col in joined_data.columns:
    if any(isinstance(val, list) for val in joined_data[col]):
        print('Column: {0}, has a list in it'.format(col))
        # Convert lists to strings
        joined_data[col] = joined_data[col].apply(lambda x: str(x) if isinstance(x, list) else x)




Column: osmid, has a list in it
Column: lanes, has a list in it
Column: highway, has a list in it
Column: maxspeed, has a list in it
Column: reversed, has a list in it
Column: name, has a list in it


In [19]:
joined_data

osmid  \
u           v           key                                                      
6565642     13800629    0                                              2953792   
6565645     13800665    0                               [921085898, 194663139]   
6565653     13801221    0                                 [921446883, 8125391]   
6565679     13800639    0    [135903555, 135903561, 38061098, 38061099, 161...   
            447588156   0                      [921061592, 921061591, 2953839]   
...                                                                        ...   
11879532996 11879532998 0                                           1279716767   
11879532998 11879532982 0                                           1279716767   
            250208162   0                                           1279716769   
11879533019 1541333358  0                                            459143891   
            2889011248  0                    [935302208, 738932787, 935302206]   

                             oneway            lanes  ref       highway  \
u           v           key                                               
6565642     13800629    0      True                3   A1         trunk   
6565645     13800665    0      True                3   A1         trunk   
6565653     13801221    0      True                1   A1    trunk_link   
6565679     13800639    0      True                2   A1         trunk   
            447588156   0      True  ['3', '1', '2']  NaN    trunk_link   
...                             ...              ...  ...           ...   
11879532996 11879532998 0      True              NaN  NaN  unclassified   
11879532998 11879532982 0      True              NaN  NaN  unclassified   
            250208162   0      True                1  NaN  unclassified   
11879533019 1541333358  0      True                2  NaN      tertiary   
            2889011248  0     False              NaN  NaN      tertiary   

                            maxspeed reversed  length_left  \
u           v           key                                  
6565642     13800629    0     60 mph    False      612.471   
6565645     13800665    0     60 mph    False      489.465   
6565653     13801221    0     70 mph    False      633.060   
6565679     13800639    0     70 mph    False     1073.101   
            447588156   0     70 mph    False      515.087   
...                              ...      ...          ...   
11879532996 11879532998 0        NaN    False        5.937   
11879532998 11879532982 0        NaN    False       10.194   
            250208162   0        NaN    False       14.815   
11879533019 1541333358  0     30 mph    False       10.365   
            2889011248  0     30 mph     True      202.516   

                                                                      geometry  \
u           v           key                                                      
6565642     13800629    0    LINESTRING (-1.68446 54.99226, -1.68660 54.990...   
6565645     13800665    0    LINESTRING (-1.67287 55.00194, -1.67570 54.999...   
6565653     13801221    0    LINESTRING (-1.69809 54.97782, -1.69796 54.977...   
6565679     13800639    0    LINESTRING (-1.62902 55.03969, -1.62883 55.039...   
            447588156   0    LINESTRING (-1.62902 55.03969, -1.62832 55.038...   
...                                                                        ...   
11879532996 11879532998 0    LINESTRING (-1.66569 55.01286, -1.66570 55.012...   
11879532998 11879532982 0    LINESTRING (-1.66574 55.01281, -1.66578 55.012...   
            250208162   0    LINESTRING (-1.66574 55.01281, -1.66578 55.012...   
11879533019 1541333358  0    LINESTRING (-1.66668 55.01330, -1.66670 55.01320)   
            2889011248  0    LINESTRING (-1.66668 55.01330, -1.66661 55.013...   

                            bridge  ... merged_data_SS_10-14A  \
u           v           key         ...                         
6565642     13800629    0 

In [17]:
output_file_path = r'C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\Experimental\ncl_OSMOS_speeds_merged.gpkg'


In [18]:
joined_data.to_file(output_file_path, driver='GPKG')